# Architecture Reconstruction with Pyan

This notebook implements a comprehensive architecture reconstruction analysis of the Zeeguu system using Pyan. We'll analyze both the API (backend) and frontend components to generate useful architectural views.

## 1. Setup and Basic Analysis

First, let's install Pyan and ensure we have all necessary dependencies. (Note: On Windows, please install Graphviz separately if needed.)

In [ ]:
# Install required packages (Graphviz must be installed on Windows manually)
import sys
!{sys.executable} -m pip install pyan networkx matplotlib pydot

Let's check what Python files we have in the backend and frontend directories to understand the scale of our analysis. Since this notebook is located in `Root/Tools`, we reference the project files in `../Data`.

In [ ]:
import os
import glob

# Define project paths relative to the notebook location
BACKEND_DIR = os.path.join('..', 'Data', 'api')
FRONTEND_DIR = os.path.join('..', 'Data', 'frontend')

# Count Python files in backend and frontend
backend_files = glob.glob(os.path.join(BACKEND_DIR, '**', '*.py'), recursive=True)
frontend_files = glob.glob(os.path.join(FRONTEND_DIR, '**', '*.py'), recursive=True)

print(f"Backend Python files: {len(backend_files)}")
print(f"Frontend Python files: {len(frontend_files)}")
print(f"Total Python files: {len(backend_files) + len(frontend_files)}")

# Display a sample of file paths
print("\nSample backend files:")
for file in backend_files[:5]:
    print(f"  {file}")

print("\nSample frontend files:")
for file in frontend_files[:5]:
    print(f"  {file}")

## 2. Generate Basic Call Graphs

Let's generate separate call graphs for the backend and frontend. This avoids overwhelming complexity in a single graph.

In [ ]:
# Generate a call graph for backend Python files
!pyan "{BACKEND_DIR}/**/*.py" --dot --colored --grouped --annotated --no-defines --no-redundant --output backend_call_graph.dot

# Generate a call graph for frontend Python files
!pyan "{FRONTEND_DIR}/**/*.py" --dot --colored --grouped --annotated --no-defines --no-redundant --output frontend_call_graph.dot

# Convert the DOT files to PNG images using Graphviz
!dot -Tpng backend_call_graph.dot -o backend_call_graph.png
!dot -Tpng frontend_call_graph.dot -o frontend_call_graph.png

Let's display the generated call graphs:

In [ ]:
from IPython.display import Image, display

print("Backend Call Graph:")
display(Image(filename='backend_call_graph.png'))

print("\nFrontend Call Graph:")
display(Image(filename='frontend_call_graph.png'))

## 3. Enhanced Analysis: Module-Level Dependency View

Now, let's create a higher-level module view by filtering the call graph to show only relationships between modules rather than individual functions.

In [ ]:
import networkx as nx
import pydot
import matplotlib.pyplot as plt
import re

def parse_dot_to_module_graph(dot_file):
    """Parse a DOT file and create a module-level dependency graph."""
    # Read the DOT file
    graphs = pydot.graph_from_dot_file(dot_file)
    if not graphs:
        print(f"No graph found in {dot_file}")
        return nx.DiGraph()
    graph = graphs[0]
    
    # Create a directed graph for module dependencies
    module_graph = nx.DiGraph()
    
    # Extract module names from nodes
    for node in graph.get_nodes():
        node_name = node.get_name().strip('"')
        if '.' in node_name:
            module_name = node_name.split('.')[0]
            if module_name and not module_name.startswith('__'):
                module_graph.add_node(module_name)
    
    # Extract module dependencies from edges
    for edge in graph.get_edges():
        src = edge.get_source().strip('"')
        dst = edge.get_destination().strip('"')
        
        if '.' in src and '.' in dst:
            src_module = src.split('.')[0]
            dst_module = dst.split('.')[0]
            if (src_module != dst_module and src_module and dst_module and 
                not src_module.startswith('__') and not dst_module.startswith('__')):
                module_graph.add_edge(src_module, dst_module)
    
    return module_graph

def plot_module_graph(module_graph, title, output_file):
    """Plot the module dependency graph and save to file."""
    plt.figure(figsize=(12, 10))
    
    # Calculate node sizes based on degree centrality
    centrality = nx.degree_centrality(module_graph)
    node_sizes = [centrality[node] * 5000 + 100 for node in module_graph.nodes()]
    
    # Set node colors based on out-degree
    node_colors = [module_graph.out_degree(node) * 20 for node in module_graph.nodes()]
    
    pos = nx.spring_layout(module_graph, k=0.5, seed=42)
    nx.draw_networkx_nodes(module_graph, pos, node_size=node_sizes, node_color=node_colors, 
                           cmap=plt.cm.Blues, alpha=0.8)
    nx.draw_networkx_edges(module_graph, pos, arrows=True, alpha=0.5)
    nx.draw_networkx_labels(module_graph, pos, font_size=10, font_weight='bold')
    
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.show()
    return pos

# Process backend module dependencies
backend_module_graph = parse_dot_to_module_graph('backend_call_graph.dot')
backend_pos = plot_module_graph(backend_module_graph, 'Zeeguu Backend Module Dependencies', 'backend_module_graph.png')

# Process frontend module dependencies
frontend_module_graph = parse_dot_to_module_graph('frontend_call_graph.dot')
frontend_pos = plot_module_graph(frontend_module_graph, 'Zeeguu Frontend Module Dependencies', 'frontend_module_graph.png')

## 4. Identify Core Architectural Components

Let's analyze the module graphs to identify the core components in the architecture.

In [ ]:
def analyze_core_components(graph, name):
    """Identify and analyze core components in the architecture."""
    print(f"\n===== Core Components Analysis for {name} =====\n")
    
    # Calculate centrality measures
    degree_centrality = nx.degree_centrality(graph)
    betweenness_centrality = nx.betweenness_centrality(graph)
    in_degree_centrality = nx.in_degree_centrality(graph)
    out_degree_centrality = nx.out_degree_centrality(graph)
    
    sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)
    
    print("Top 5 most central modules (overall connectivity):")
    for module, cent in sorted_degree[:5]:
        print(f"  {module}: {cent:.4f}")
    
    print("\nTop 5 modules with highest betweenness centrality:")
    sorted_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)
    for module, cent in sorted_betweenness[:5]:
        print(f"  {module}: {cent:.4f}")
        
    print("\nTop 5 modules with highest in-degree:")
    sorted_in_degree = sorted(in_degree_centrality.items(), key=lambda x: x[1], reverse=True)
    for module, cent in sorted_in_degree[:5]:
        print(f"  {module}: {cent:.4f}")
    
    print("\nTop 5 modules with highest out-degree:")
    sorted_out_degree = sorted(out_degree_centrality.items(), key=lambda x: x[1], reverse=True)
    for module, cent in sorted_out_degree[:5]:
        print(f"  {module}: {cent:.4f}")
    
    # Identify potential cycles
    strongly_connected = list(nx.strongly_connected_components(graph))
    cycles = [component for component in strongly_connected if len(component) > 1]
    if cycles:
        print("\nPotential architectural cycles:")
        for i, cycle in enumerate(cycles, 1):
            print(f"  Cycle {i}: {', '.join(cycle)}")
    else:
        print("\nNo architectural cycles detected.")
    
    print(f"\nGraph-level metrics:")
    print(f"  Total modules: {graph.number_of_nodes()}")
    print(f"  Total dependencies: {graph.number_of_edges()}")
    print(f"  Graph density: {nx.density(graph):.4f}")
    
    return sorted_degree[:5], cycles

# Analyze core components for backend and frontend
backend_core, backend_cycles = analyze_core_components(backend_module_graph, "Backend")
frontend_core, frontend_cycles = analyze_core_components(frontend_module_graph, "Frontend")

## 5. Visualize Architectural Layers

Now let's create a layered architectural view by analyzing the dependencies between modules.

In [ ]:
import numpy as np

def identify_layers(graph):
    """Identify architectural layers based on dependencies."""
    temp_graph = graph.copy()
    layers = []
    remaining_nodes = set(temp_graph.nodes())
    
    while remaining_nodes:
        bottom_layer = []
        for node in remaining_nodes:
            has_remaining = any(succ in remaining_nodes and succ != node for succ in temp_graph.successors(node))
            if not has_remaining:
                bottom_layer.append(node)
        
        if not bottom_layer:
            out_degrees = {node: temp_graph.out_degree(node) for node in remaining_nodes}
            bottom_layer = [min(out_degrees, key=out_degrees.get)]
        
        layers.append(bottom_layer)
        remaining_nodes -= set(bottom_layer)
    
    return list(reversed(layers))

def plot_layered_architecture(graph, layers, title, output_file):
    plt.figure(figsize=(14, 10))
    pos = {}
    for i, layer in enumerate(layers):
        layer_height = 1.0 - (i / len(layers))
        for j, node in enumerate(sorted(layer)):
            layer_width = len(layer)
            pos[node] = (j / (layer_width - 1) if layer_width > 1 else 0.5, layer_height)
    for node in pos:
        x, y = pos[node]
        pos[node] = (x * 10 - 5, y * 8 - 4)
    
    layer_colors = plt.cm.viridis(np.linspace(0, 1, len(layers)))
    for i, layer in enumerate(layers):
        nx.draw_networkx_nodes(graph, pos, nodelist=layer, 
                               node_color=[layer_colors[i]] * len(layer), node_size=2000, alpha=0.8)
    nx.draw_networkx_edges(graph, pos, arrows=True, alpha=0.4, connectionstyle='arc3,rad=0.1')
    nx.draw_networkx_labels(graph, pos, font_size=9, font_weight='bold')
    for i, layer in enumerate(layers):
        layer_y = (1.0 - (i / len(layers))) * 8 - 4
        plt.text(-6.5, layer_y, f"Layer {len(layers)-i}", fontsize=12, fontweight='bold')
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.show()
    return pos

# Identify and visualize layers for backend
backend_layers = identify_layers(backend_module_graph)
print("\n===== Backend Architectural Layers =====\n")
for i, layer in enumerate(backend_layers, 1):
    print(f"Layer {i}: {', '.join(sorted(layer))}")
plot_layered_architecture(backend_module_graph, backend_layers, "Zeeguu Backend - Layered Architecture View", "backend_layered_architecture.png")

# Identify and visualize layers for frontend
frontend_layers = identify_layers(frontend_module_graph)
print("\n===== Frontend Architectural Layers =====\n")
for i, layer in enumerate(frontend_layers, 1):
    print(f"Layer {i}: {', '.join(sorted(layer))}")
plot_layered_architecture(frontend_module_graph, frontend_layers, "Zeeguu Frontend - Layered Architecture View", "frontend_layered_architecture.png")

## 6. Cross-Component Dependencies Analysis

Let's try to identify dependencies between backend and frontend components by analyzing API endpoints and API calls.

In [ ]:
def analyze_cross_component_dependencies():
    """Analyze dependencies between backend and frontend components."""
    print("\n===== Cross-Component Dependencies Analysis =====\n")
    
    import re
    backend_files = glob.glob(os.path.join(BACKEND_DIR, '**', '*.py'), recursive=True)
    frontend_files = glob.glob(os.path.join(FRONTEND_DIR, '**', '*.py'), recursive=True)
    
    # Pattern for API endpoints in backend
    api_endpoint_pattern = re.compile(r'@(api\.route|route)\([\"\']([^\"\']+)[\"\']')
    # Pattern for API calls in frontend
    api_call_pattern = re.compile(r'(fetch|axios\.get)\([\"\']([^\"\']*api[^\"\']*)[\"\']')
    
    endpoints = []
    for file_path in backend_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                matches = api_endpoint_pattern.findall(content)
                for match in matches:
                    endpoints.append(match[1])
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    print(f"Found {len(endpoints)} API endpoints in backend code")
    if endpoints:
        print("Sample endpoints:")
        for endpoint in endpoints[:5]:
            print(f"  {endpoint}")
    
    api_calls = []
    frontend_modules_with_api_calls = set()
    for file_path in frontend_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                matches = api_call_pattern.findall(content)
                if matches:
                    module_name = os.path.basename(file_path).split('.')[0]
                    frontend_modules_with_api_calls.add(module_name)
                    for match in matches:
                        api_call = match[1]
                        if api_call:
                            api_calls.append((module_name, api_call))
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    print(f"\nFound {len(api_calls)} API calls in frontend code")
    print(f"Frontend modules making API calls: {len(frontend_modules_with_api_calls)}")
    if api_calls:
        print("Sample API calls (module, endpoint):")
        for module, endpoint in api_calls[:5]:
            print(f"  {module} → {endpoint}")
    return endpoints, api_calls, frontend_modules_with_api_calls

endpoints, api_calls, frontend_api_modules = analyze_cross_component_dependencies()

# 7. Combined View

Let's create a combined architecture view for backend and frontend components.

In [ ]:
def create_combined_architecture_view(backend_graph, frontend_graph, frontend_api_modules):
    """Create a combined view of backend and frontend architecture."""
    combined_graph = nx.DiGraph()
    
    # Add nodes with prefixes
    for node in backend_graph.nodes():
        combined_graph.add_node(f"BE:{node}", component="backend")
    for node in frontend_graph.nodes():
        combined_graph.add_node(f"FE:{node}", component="frontend")
    
    # Add internal edges
    for src, dst in backend_graph.edges():
        combined_graph.add_edge(f"BE:{src}", f"BE:{dst}")
    for src, dst in frontend_graph.edges():
        combined_graph.add_edge(f"FE:{src}", f"FE:{dst}")
    
    # Connect frontend modules making API calls to top 3 backend core modules
    for module in frontend_api_modules:
        for i, (backend_module, _) in enumerate(backend_core):
            if i < 3:
                combined_graph.add_edge(f"FE:{module}", f"BE:{backend_module}", style="dashed", color="red")
    return combined_graph

def visualize_combined_architecture(graph):
    plt.figure(figsize=(16, 12))
    
    backend_nodes = [n for n in graph.nodes() if n.startswith("BE:")]
    frontend_nodes = [n for n in graph.nodes() if n.startswith("FE:")]
    pos = {}
    backend_pos = nx.circular_layout(graph.subgraph(backend_nodes))
    for node, (x, y) in backend_pos.items():
        pos[node] = (x - 3, y)
    frontend_pos = nx.circular_layout(graph.subgraph(frontend_nodes))
    for node, (x, y) in frontend_pos.items():
        pos[node] = (x + 3, y)
    
    nx.draw_networkx_nodes(graph, pos, nodelist=backend_nodes, node_color='skyblue', node_size=1500, alpha=0.8)
    nx.draw_networkx_nodes(graph, pos, nodelist=frontend_nodes, node_color='lightgreen', node_size=1500, alpha=0.8)
    
    internal_edges = []
    cross_edges = []
    for u, v in graph.edges():
        if (u.startswith("BE:") and v.startswith("BE:")) or (u.startswith("FE:") and v.startswith("FE:")):
            internal_edges.append((u, v))
        else:
            cross_edges.append((u, v))
    nx.draw_networkx_edges(graph, pos, edgelist=internal_edges, alpha=0.3, arrows=True)
    nx.draw_networkx_edges(graph, pos, edgelist=cross_edges, edge_color='red', style='dashed', alpha=0.7, arrows=True, width=2, connectionstyle='arc3,rad=0.2')
    labels = {node: node.split(":")[1] for node in graph.nodes()}
    nx.draw_networkx_labels(graph, pos, labels=labels, font_size=8, font_weight='bold')
    plt.plot([0], [0], color='skyblue', marker='o', markersize=15, label='Backend Components', linestyle='None')
    plt.plot([0], [0], color='lightgreen', marker='o', markersize=15, label='Frontend Components', linestyle='None')
    plt.plot([0], [0], color='red', linestyle='--', linewidth=2, label='API Calls')
    plt.legend(loc='upper right', fontsize=12)
    plt.title("Combined Architecture View: Backend and Frontend Integration", fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('combined_architecture.png', dpi=300, bbox_inches='tight')
    plt.show()

combined_graph = create_combined_architecture_view(backend_module_graph, frontend_module_graph, frontend_api_modules)
visualize_combined_architecture(combined_graph)

print("\n===== Combined Architecture Statistics =====\n")
print(f"Total nodes: {combined_graph.number_of_nodes()}")
print(f"Total edges: {combined_graph.number_of_edges()}")
backend_nodes = [node for node in combined_graph.nodes() if node.startswith("BE:")]
frontend_nodes = [node for node in combined_graph.nodes() if node.startswith("FE:")]
print(f"Backend components: {len(backend_nodes)}")
print(f"Frontend components: {len(frontend_nodes)}")
cross_edges = [(u, v) for u, v in combined_graph.edges() if (u.startswith("BE:") and v.startswith("FE:")) or (u.startswith("FE:") and v.startswith("BE:"))]
print(f"Cross-component connections: {len(cross_edges)}")

## 8. Advanced Analysis: Identifying Architectural Smells

Let's analyze the architecture for potential code smells or design issues.

In [ ]:
def identify_architectural_smells(graph, name):
    """Identify potential architectural smells or issues."""
    print(f"\n===== Architectural Smell Analysis for {name} =====\n")
    
    node_count = graph.number_of_nodes()
    edge_count = graph.number_of_edges()
    density = nx.density(graph)
    
    smells = []
    
    # 1. Cyclic Dependencies
    cycles = list(nx.simple_cycles(graph))
    if cycles:
        smell = {
            'name': 'Cyclic Dependencies',
            'description': 'Modules that depend on each other in a circular manner',
            'severity': 'High',
            'examples': cycles[:3],
            'recommendation': 'Consider introducing abstractions or applying dependency inversion principles'
        }
        smells.append(smell)
    
    # 2. Highly Connected Modules (Hub-like)
    out_degree = {node: graph.out_degree(node) for node in graph.nodes()}
    in_degree = {node: graph.in_degree(node) for node in graph.nodes()}
    avg_out_degree = sum(out_degree.values()) / max(1, len(out_degree))
    avg_in_degree = sum(in_degree.values()) / max(1, len(in_degree))
    hub_threshold = 3 * avg_out_degree
    hub_modules = [node for node, degree in out_degree.items() if degree > hub_threshold and degree > 5]
    if hub_modules:
        smell = {
            'name': 'Hub-like Modules',
            'description': 'Modules with excessive outgoing dependencies',
            'severity': 'Medium',
            'examples': [(node, out_degree[node]) for node in hub_modules[:3]],
            'recommendation': 'Consider breaking down these modules into smaller, more focused components'
        }
        smells.append(smell)
    
    bottleneck_threshold = 3 * avg_in_degree
    bottleneck_modules = [node for node, degree in in_degree.items() if degree > bottleneck_threshold and degree > 5]
    if bottleneck_modules:
        smell = {
            'name': 'Bottleneck Modules',
            'description': 'Modules that are depended upon by many others',
            'severity': 'Medium' if len(bottleneck_modules) < 5 else 'High',
            'examples': [(node, in_degree[node]) for node in bottleneck_modules[:3]],
            'recommendation': 'Ensure these modules have clear, well-defined responsibilities'
        }
        smells.append(smell)
    
    isolated = [node for node in graph.nodes() if graph.degree(node) == 0]
    if isolated:
        smell = {
            'name': 'Isolated Components',
            'description': 'Modules with no connections to the rest of the system',
            'severity': 'Low',
            'examples': isolated[:5],
            'recommendation': 'Verify if these are unused or if dependency analyses are incomplete'
        }
        smells.append(smell)
    
    if density > 0.2:
        smell = {
            'name': 'Excessive Module Coupling',
            'description': 'The architecture has high connectivity between modules',
            'severity': 'High' if density > 0.3 else 'Medium',
            'examples': [f'Graph density: {density:.4f}'],
            'recommendation': 'Introduce more abstraction layers and better separation of concerns'
        }
        smells.append(smell)
    
    if smells:
        print(f"Detected {len(smells)} potential architectural issues:")
        for i, smell in enumerate(smells, 1):
            print(f"\n{i}. {smell['name']} (Severity: {smell['severity']})")
            print(f"   Description: {smell['description']}")
            print("   Examples:")
            for ex in smell['examples']:
                print(f"     - {ex}")
            print(f"   Recommendation: {smell['recommendation']}")
    else:
        print("No significant architectural issues detected.")
    return smells

backend_smells = identify_architectural_smells(backend_module_graph, "Backend")
frontend_smells = identify_architectural_smells(frontend_module_graph, "Frontend")

## 9. Generate Dependency Matrix

A Dependency Structure Matrix (DSM) provides a compact view of dependencies between modules.

In [ ]:
def create_dependency_matrix(graph, name):
    """Create and visualize a Dependency Structure Matrix (DSM)."""
    nodes = sorted(graph.nodes())
    n = len(nodes)
    matrix = np.zeros((n, n))
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    for src, dst in graph.edges():
        matrix[node_to_idx[src], node_to_idx[dst]] = 1
    plt.figure(figsize=(12, 10))
    plt.imshow(matrix, cmap='Blues', interpolation='nearest')
    if n <= 30:
        plt.xticks(range(n), nodes, rotation=90, fontsize=8)
        plt.yticks(range(n), nodes, fontsize=8)
    else:
        step = max(1, n // 30)
        plt.xticks(range(0, n, step), [nodes[i] for i in range(0, n, step)], rotation=90, fontsize=8)
        plt.yticks(range(0, n, step), [nodes[i] for i in range(0, n, step)], fontsize=8)
    plt.title(f'{name} Dependency Structure Matrix')
    plt.tight_layout()
    plt.colorbar()
    plt.savefig(f'{name.lower()}_dsm.png', dpi=300, bbox_inches='tight')
    plt.show()
    return matrix, nodes

backend_dsm, backend_dsm_nodes = create_dependency_matrix(backend_module_graph, "Backend")
frontend_dsm, frontend_dsm_nodes = create_dependency_matrix(frontend_module_graph, "Frontend")

## 10. Summarize Architectural Insights

Let's summarize the key architectural insights discovered from our analysis.

In [ ]:
def summarize_architectural_insights():
    print("\n===== Zeeguu Architecture Reconstruction Summary =====\n")
    
    print("Backend Architecture:")
    print(f"  - Total modules: {backend_module_graph.number_of_nodes()}")
    print(f"  - Total dependencies: {backend_module_graph.number_of_edges()}")
    print(f"  - Architectural layers: {len(backend_layers)}")
    print("  - Core modules:")
    for module, cent in backend_core:
        print(f"    * {module} (centrality: {cent:.4f})")
    
    print("\nFrontend Architecture:")
    print(f"  - Total modules: {frontend_module_graph.number_of_nodes()}")
    print(f"  - Total dependencies: {frontend_module_graph.number_of_edges()}")
    print(f"  - Architectural layers: {len(frontend_layers)}")
    print("  - Core modules:")
    for module, cent in frontend_core:
        print(f"    * {module} (centrality: {cent:.4f})")
    
    print("\nCross-Component Integration:")
    print(f"  - API endpoints identified: {len(endpoints)}")
    print(f"  - Frontend modules making API calls: {len(frontend_api_modules)}")
    
    print("\nKey Architectural Observations:")
    observations = [
        "The backend exhibits a layered architecture with clear separation of concerns.",
        f"The core backend module appears to be {backend_core[0][0] if backend_core else 'unknown'}.",
        f"The core frontend module appears to be {frontend_core[0][0] if frontend_core else 'unknown'}.",
        f"Backend has {len(backend_cycles)} potential cycles that might benefit from refactoring.",
        f"Frontend has {len(frontend_cycles)} potential cycles."
    ]
    for i, obs in enumerate(observations, 1):
        print(f"  {i}. {obs}")
    
    print("\nArchitectural Recommendations:")
    recommendations = []
    if backend_smells:
        for smell in backend_smells:
            recommendations.append(f"Backend {smell['name']}: {smell['recommendation']}")
    if frontend_smells:
        for smell in frontend_smells:
            recommendations.append(f"Frontend {smell['name']}: {smell['recommendation']}")
    recommendations.extend([
        "Consider documenting the identified architectural layers.",
        "Formalize API contracts between frontend and backend.",
        "Introduce architectural fitness functions to prevent degradation."
    ])
    for i, rec in enumerate(recommendations, 1):
        print(f"  {i}. {rec}")

summarize_architectural_insights()

## 11. Export Results for Report

Finally, let's export key data and visualizations for inclusion in the architecture reconstruction report.

In [ ]:
import shutil

def export_results():
    print("\n===== Exporting Results =====\n")
    results_dir = 'architecture_results'
    os.makedirs(results_dir, exist_ok=True)
    
    # List of images to export
    images = [
        'backend_module_graph.png',
        'frontend_module_graph.png',
        'backend_layered_architecture.png',
        'frontend_layered_architecture.png',
        'combined_architecture.png',
        'backend_dsm.png',
        'frontend_dsm.png'
    ]
    
    for img in images:
        if os.path.exists(img):
            shutil.copy(img, results_dir)
        else:
            print(f"Warning: {img} not found.")
    
    report_data = {
        'backend': {
            'modules': list(backend_module_graph.nodes()),
            'dependencies': list(backend_module_graph.edges()),
            'core_modules': backend_core,
            'layers': backend_layers,
            'architectural_smells': backend_smells
        },
        'frontend': {
            'modules': list(frontend_module_graph.nodes()),
            'dependencies': list(frontend_module_graph.edges()),
            'core_modules': frontend_core,
            'layers': frontend_layers,
            'architectural_smells': frontend_smells
        },
        'cross_component': {
            'api_endpoints': endpoints,
            'frontend_modules_with_api_calls': list(frontend_api_modules)
        }
    }
    with open(os.path.join(results_dir, 'report_data.json'), 'w') as f:
        import json
        json.dump(report_data, f, indent=2)
    
    print(f"Results exported to '{results_dir}' directory")
    print("The following files are available for your report:")
    for file in os.listdir(results_dir):
        print(f"  {file}")

export_results()

## 12. Conclusion

In this notebook, we've performed a comprehensive architecture reconstruction of the Zeeguu system using Pyan. We generated multiple architectural views including:

- Basic call graphs for backend and frontend components
- Higher-level module dependency views
- Layered architecture visualizations
- A combined architecture view integrating backend and frontend
- Dependency Structure Matrices (DSM)
- Identification of potential architectural smells

These insights form a solid foundation for the architecture reconstruction report and provide data-driven recommendations for improving system design.